In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
import numpy as np
import os
import create_directory

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [8]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(os.path.join(create_directory.marvel_data_dir, "preprocess/Train/mtcnn"), transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

In [5]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [7]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])
batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [9]:
train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [10]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [11]:
print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
    )



Initial
----------
Valid |     7/7    | loss:    1.7390 | fps:    3.5268 | acc:    0.3304   

Epoch 1/8
----------
Train |    27/27   | loss:    0.7319 | fps:    1.2795 | acc:    0.7500   
Valid |     7/7    | loss:    1.1517 | fps:    3.3862 | acc:    0.7402   

Epoch 2/8
----------
Train |    27/27   | loss:    0.3423 | fps:    1.6107 | acc:    0.8819   
Valid |     7/7    | loss:    0.7705 | fps:    3.1054 | acc:    0.8152   

Epoch 3/8
----------
Train |    27/27   | loss:    0.1586 | fps:    1.2836 | acc:    0.9456   
Valid |     7/7    | loss:    0.3983 | fps:    3.3086 | acc:    0.8696   

Epoch 4/8
----------
Train |    27/27   | loss:    0.1017 | fps:    1.1593 | acc:    0.9745   
Valid |     7/7    | loss:    0.2583 | fps:    2.6236 | acc:    0.9214   

Epoch 5/8
----------
Train |    11/27   | loss:    0.0826 | fps:    1.0668 | acc:    0.9773   

In [ ]:
torch.save({'model': resnet.state_dict(),
            'optim': optimizer.state_dict(),
           }, os.path.join(create_directory.recognition_model_dir, "facenet_finetune.pth"))